In [1]:
import pandas as pd
import numpy as np
import os
import re
import datetime
import glob
from tqdm.notebook import tqdm
from sklearn.neighbors import BallTree
from lotek.gps import calc_ta, calc_dist
from lotek.conversion import shp2mask

In [27]:
"""
Define files to be used
"""
# CSV of GPS collar data
#gps_f = 'C:/SPK_local/data/cattle_gps/Lotek/2017/TRMappended2017_GoodDays_TrackQ3.csv'
inDIR = './Moonitor_DailyMetrics/12-2-2022/'
outDIR = './Moonitor_DailyMetrics/processed'
out_f = 'moonitor_gps_2019_D1_Q1.csv'

In [28]:
gps_f_LIST = glob.glob(inDIR + '*.csv')
if not os.path.exists(outDIR):
    os.mkdir(outDIR)

In [29]:
"""
Read in GPS data and format
"""
# read data
df_gps = pd.DataFrame()
for f in tqdm(gps_f_LIST):
    df_gps_tmp = pd.read_csv(f, engine='c', parse_dates=['DateTime', 'date'])
    df_gps_tmp['date'] = df_gps_tmp['date'].dt.date
    df_gps_tmp['Steer_ID'] = os.path.basename(f).split('_')[0]
    df_gps = pd.concat([df_gps, df_gps_tmp])
#df_gps = pd.read_csv(gps_f, engine='c')

  0%|          | 0/11 [00:00<?, ?it/s]

In [30]:
df_gps.dtypes

Unnamed: 0                int64
DateTime         datetime64[ns]
Lat                     float64
Long                    float64
date                     object
time                     object
rest                      int64
graze                     int64
walk                      int64
sum                       int64
timediff_mins             int64
distprev                float64
distnext                float64
bearingprev             float64
bearingnext             float64
turnangle               float64
UTM_X                   float64
UTM_Y                   float64
distAB                  float64
distBC                  float64
distAC                  float64
turnangleUTM            float64
PctGrz                  float64
PctRest                 float64
GrzBin                    int64
velo_mpmin              float64
velo_flag                 int64
ta_flag                   int64
veloflag2                 int64
tot_flag                  int64
DateTime_Char            object
Steer_ID

In [31]:
# fix column names as needed
#col_remap_dict = {
#    'Date_Time_Fix': 'Date_Time',
#    'duration_check': 'duration'
#}
#df_gps = df_gps.rename(columns=col_remap_dict)

# convert datetime strings to dates
#df_gps['Fix_DateTime'] = pd.to_datetime(df_gps['date'].astype(str) + ' ' + df_gps['time'])
#df_gps['Fix_Date'] = pd.to_datetime(df_gps['Fix_DateTime'].dt.date)

# sort data by fix time for each steer
df_gps = df_gps.groupby('Steer_ID').apply(lambda x: x.sort_values('DateTime')).reset_index(drop=True)

# calculate the actual duration between fixes
df_gps['Fix_Duration'] = df_gps.groupby('Steer_ID').apply(
    lambda x: (x.DateTime - x.DateTime.shift(1)).astype("timedelta64[s]")).reset_index()['DateTime'] / 60.0

In [32]:
"""
Clean up data based on grazing hrs per day and missing fixes per day
"""
# define minimum and maximum grazing hours per day for removing days from GPS dataset
hrs_min = 6
hrs_max = 13

# define maximum number of missing fixes allowed per day
missing_max = 20

In [33]:
"""
Calculate movment stats at each GPS fix
    turning angle (degrees): angle between previous, current and next fix, 
        converted to difference from straight line (0) with possible values ranging 0-180
    step length (m): distance from previous to current fix
    movement rate (m/min): distance (m) between previous and current fix divided by
        time (mins) between previous and current fix
"""

# calculate the turning angle and distance at each fix for each day and each steer
df_gps['steplength'] = np.nan
df_gps['turnangle'] = np.nan
for group in tqdm(df_gps.groupby(['Steer_ID', 'date'])):
    group[1]['UTM_X_lag1'] = group[1]['UTM_X'].shift(1)
    group[1]['UTM_Y_lag1'] = group[1]['UTM_Y'].shift(1)
    group[1]['UTM_X_lead1'] = group[1]['UTM_X'].shift(-1)
    group[1]['UTM_Y_lead1'] = group[1]['UTM_Y'].shift(-1)
    a_list = list(group[1][['UTM_X_lag1', 'UTM_Y_lag1']].values)
    b_list = list(group[1][['UTM_X', 'UTM_Y']].values)
    c_list = list(group[1][['UTM_X_lead1', 'UTM_Y_lead1']].values)
    dist_mask = ~(np.any(np.isnan(a_list), axis=1) |  np.any(np.isnan(b_list), axis=1))
    sl_tmp = np.ones_like(dist_mask) * np.nan
    sl_tmp[dist_mask] = calc_dist(np.array(list(map(tuple, a_list)))[dist_mask], 
                                  np.array(list(map(tuple, b_list)))[dist_mask]).squeeze()
    df_gps.loc[(df_gps['Steer_ID'] == group[0][0]) & (df_gps['date'] == group[0][1]),
               'steplength'] = sl_tmp
    df_gps.loc[(df_gps['Steer_ID'] == group[0][0]) & (df_gps['date'] == group[0][1]),
               'turnangle'] = calc_ta(a_list, b_list, c_list)

# Calculate movement rate from distance and timestamp
df_gps['moverate'] = df_gps['steplength'] / df_gps['Fix_Duration']

  0%|          | 0/224 [00:00<?, ?it/s]

In [34]:
"""
Create flags for data to be removed/masked
    * jump_flag: suspected jump based on movement rate > 42 m/min and turn angle > 120 degrees
    * fast_flag: suspected error based on movement rate > 84 m/min
    * missingfix_flag: flag for entire day’s fixes based on > 20 missing 5-min fixes for the day
    * badfix_flag: flag indicating if any of the previous three flags exist. 
      This is used for calculating grazing hrs per day (if any of the previous three flags are present, the fix is not included when calculating grazing hours per day)
    * grazinghrs_flag: flag for the entire day’s fixes based on grazing hours < 6 or > 13.
"""
# flag all locations suspected as jumps (movement rate > 42 m/min and turnangle > 120 degrees)
jump_flag = (df_gps['moverate'] > 42) & (df_gps['turnangle'] > 120)
df_gps['jump_flag'] = jump_flag.astype(int)

# flag all locations with movement rate > 84 m/min while grazing
fast_flag = (df_gps['moverate'] > 84) & (df_gps['GrzBin'] == 1)
df_gps['fast_flag'] = fast_flag.astype(int)

# flag all days with more than the maximum number of allowed missing fixes
missingfix_flag = df_gps.groupby(['date', 'Steer_ID'])['GrzBin'].transform('count') < (24 * (60 / 5) - missing_max)
df_gps['missingfix_flag'] = missingfix_flag.astype(int)

# combine the three masks above to flag any data that should not be included when calculating grazing hours
badfix_flag = jump_flag | fast_flag | missingfix_flag
df_gps['badfix_flag'] = badfix_flag.astype(int)

# calculate total time in hrs spent grazing daily
df_gps['grazing_hrs'] = df_gps[~badfix_flag].groupby(['date', 'Steer_ID'])['GrzBin'].transform('sum') * 5 / 60

# flag all days with less than 6 hrs and more than 13 hrs grazing
grazehrs_flag = (df_gps['grazing_hrs'] < hrs_min) | (df_gps['grazing_hrs'] > hrs_max)
df_gps['grazehrs_flag'] = grazehrs_flag.astype(int)

In [36]:
# Print the proportion of the full dataset that was removed during final cleaning 
print('--- Overall flags ---')
print('Jump flagged: ' + str(round(100*sum(jump_flag)/len(df_gps), 1)) + '%')
print('Fast fix flagged: ' + str(round(100*sum(fast_flag)/len(df_gps), 1)) + '%')
print('Missing fix flagged: ' + str(round(100*sum(missingfix_flag)/len(df_gps), 1)) + '%')
print('Grazing hrs flagged: ' + str(round(100*sum(grazehrs_flag)/len(df_gps), 1)) + '%')
print('Total flagged: ' + str(round(100*sum((badfix_flag | grazehrs_flag))/len(df_gps), 1)) + '%')

--- Overall flags ---
Jump flagged: 0.1%
Fast fix flagged: 0.0%
Missing fix flagged: 7.4%
Grazing hrs flagged: 4.6%
Total flagged: 12.1%


In [23]:
print('--- Steer-level flags ---')
df_gps.groupby('Steer_ID').apply(lambda x: x['missingfix_flag'].sum()/x['missingfix_flag'].count())

--- Steer-level flags ---


Steer_ID
d03    0.060800
d04    0.076065
d06    0.141523
d10    0.067422
d14    0.063937
d15    0.062478
d17    0.062774
d18    0.069379
d19    0.111983
d24    0.064060
d26    0.065031
dtype: float64

In [24]:
"""
Save a copy of the data to .csv that retains all data and flags
"""
# copy data
df_gps_flagged = df_gps.copy()
# write to .csv with suffix '_flagged'
df_gps_flagged.to_csv(os.path.join(outDIR, re.sub('.csv', '_flagged.csv', out_f)), index=False)

# remove all flagged data from the final cleaned dataset
df_gps = df_gps[~(badfix_flag | grazehrs_flag)].copy()

In [25]:
"""
Create activity bouts and daily activity budgets from the cleaed data
"""
# change bouts when grazing activity changes, unless the two fixes before and two fixes after are the same
# in detail: change to new bout if:
#   * activity is not the same as the previous row
#   * AND activity is the same as one of the next two rows
#   * AND activitiy is not the same as one of the two rows after it
#   * NOTE: this only works for bouts of 4+ fixes. We manually classify bouts <= 3 fixes as 'Transition' bout
df_gps['grazing_bout'] = df_gps.groupby(['date', 'Steer_ID'])['GrzBin'].apply(
    lambda x: (((x != x.shift(1)) &
                ((x == x.shift(-1)) | (x == x.shift(-2))) & 
                ((x != x.shift(2)) | (x != x.shift(3))))).cumsum())

# calculate duration of each bout in minutes
df_gps['bout_mins'] = df_gps.groupby(['date', 
                                      'Steer_ID', 
                                      'grazing_bout'])['DateTime'].transform(lambda x: (x.max() - x.min()).seconds/60 + 5.0)

# calculate the majority grazing activity for each bout to calculate bout activity
df_gps['bout_maj'] = df_gps.groupby(['date', 
                                     'Steer_ID', 
                                     'grazing_bout'])['GrzBin'].transform(lambda x: x.value_counts().index[0])

# create a bout activity column
bout_act_dict = {0: 'Nongrazing',
                1: 'Grazing'}
df_gps['bout_act'] = df_gps['bout_maj'].apply(lambda x: bout_act_dict[x])
df_gps.loc[df_gps['bout_mins'] < 20, 'bout_act'] = 'Transition'

# calculate the number of bouts per day in each activity
df_gps['act_bout_ct_daily'] = df_gps.groupby(['date', 
                                              'Steer_ID',
                                              'bout_act'])['grazing_bout'].transform('nunique')

# calculate grazing activity budgets for each day and steer
df_gps['act_budget_daily'] = df_gps.groupby(['date', 
                                             'Steer_ID',
                                             'bout_act'])['bout_mins'].transform('sum') / df_gps.groupby(['date',
                                                                                                          'Steer_ID'])['bout_mins'].transform('sum')

In [26]:
# Write data to .csv with the suffix '_cleaned.csv'
df_gps.to_csv(os.path.join(outDIR, re.sub('.csv', '_cleaned.csv', out_f)), index=False)